In [9]:
import pandas as pd
import os

In [130]:
results_total = pd.DataFrame()
for file in os.listdir():
    if pd.Series(file).str.contains('Results')[0]:
        results = pd.read_csv(file)
        results['file'] = str(file)
        results_total = pd.concat([results_total,results])

model = results_total['Unnamed: 0'].str.split('-',expand = True)
model.columns = ['Variable', 'Model_Info', 'Threshold', 'Execution Time (s)']
model['Execution Time (s)'] = model['Execution Time (s)'].str.split(':',expand=True)[1]
model['Threshold'] = model['Threshold'].str.split(' * ',expand=True)[4]
model['Model'] = model['Model_Info'].str.split('(',expand=True)[0]
model['Contamination'] = model['Model_Info'].str.split('(',expand=True)[1].str.split('contamination=',expand=True)[1].str.split(',',expand=True)[0]
model['n_neighbors'] = model['Model_Info'].str.split('(',expand=True)[1].str.split('n_neighbors=',expand=True)[1].str.split(',',expand=True)[0].str.split(')',expand=True)[0]
model['n_estimators'] = model['Model_Info'].str.split('(',expand=True)[1].str.split('n_estimators=',expand=True)[1].str.split(',',expand=True)[0]

results_total = pd.concat([results_total,model], axis=1)

results_total['Model Type'] = results_total.file.str.split('_',expand=True)[0].str.split('Results',expand=True)[0]

results_total['Training %'] = results_total.file.str.split('_',expand=True)[1].str.split('.',expand=True)[0]

results_total = results_total[['Variable','Model', 'Model_Info', 'Model Type', 'Training %','Precision', 'Recall', 'Accuracy', 'F1-score', 'AUC',
       'Train Ratio', 'Number of true anomalies',
       'Number of predicted anomalies', 'True Positives', 'False Positives',
       'True Negatives', 'False Negatives',
       'Threshold', 'Execution Time (s)', 'Contamination',
       'n_neighbors', 'n_estimators']]

In [138]:
results_total

,Variable,Model,Model_Info,Model Type,Training %,Precision,Recall,Accuracy,F1-score,AUC,...,Number of predicted anomalies,True Positives,False Positives,True Negatives,False Negatives,Threshold,Execution Time (s),Contamination,n_neighbors,n_estimators
0,DailyYield_B (KW),LOF,"LOF(algorithm='auto', contamination=0.001, le...",SemiSupervised,70,0.0,0.00,0.98,0.0,0.49,...,1814,0,1814,76718,35,11.95,0.25126099586486816,0.001,10,None
1,DailyYield_B (KW),LOF,"LOF(algorithm='auto', contamination=0.005, le...",SemiSupervised,70,0.0,0.00,0.97,0.0,0.49,...,2102,0,2102,76430,35,9.75,0.249678373336792,0.005,10,None
2,DailyYield_B (KW),LOF,"LOF(algorithm='auto', contamination=0.01, lea...",SemiSupervised,70,0.0,0.00,0.97,0.0,0.48,...,2366,0,2366,76166,35,5.47,0.24369335174560547,0.01,10,None
3,DailyYield_B (KW),LOF,"LOF(algorithm='auto', contamination=0.05, lea...",SemiSupervised,70,0.0,0.09,0.93,0.0,0.51,...,5506,3,5503,73029,32,0.0,0.24146556854248047,0.05,10,None
4,DailyYield_B (KW),LOF,"LOF(algorithm='auto', contamination=0.1, leaf...",SemiSupervised,70,0.0,0.20,0.88,0.0,0.54,...,9764,7,9757,68775,28,0.0,0.2272343635559082,0.1,10,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,AC_Power_B (KW),ECOD,"ECOD(contamination=0.005, n_jobs=1)",Unsupervised,90,0.0,0.57,0.91,0.0,0.74,...,2355,4,2351,23831,3,7.0,0.1306140422821045,0.005,None,None
116,AC_Power_B (KW),ECOD,"ECOD(contamination=0.01, n_jobs=1)",Unsupervised,90,0.0,0.57,0.91,0.0,0.74,...,2355,4,2351,23831,3,6.21,0.14786458015441895,0.01,None,None
117,AC_Power_B (KW),ECOD,"ECOD(contamination=0.05, n_jobs=1)",Unsupervised,90,0.0,0.57,0.91,0.0,0.74,...,2355,4,2351,23831,3,3.87,0.12284278869628906,0.05,None,None
118,AC_Power_B (KW),ECOD,"ECOD(contamination=0.1, n_jobs=1)",Unsupervised,90,0.0,0.57,0.91,0.0,0.74,...,2355,4,2351,23831,3,2.89,0.12979459762573242,0.1,None,None
